In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
sess = tf.Session()

In [3]:
mnist = input_data.read_data_sets("/JupyterNotebook/Machine Learning/CNN Demo", one_hot = True)

Extracting /JupyterNotebook/Machine Learning/CNN Demo\train-images-idx3-ubyte.gz
Extracting /JupyterNotebook/Machine Learning/CNN Demo\train-labels-idx1-ubyte.gz
Extracting /JupyterNotebook/Machine Learning/CNN Demo\t10k-images-idx3-ubyte.gz
Extracting /JupyterNotebook/Machine Learning/CNN Demo\t10k-labels-idx1-ubyte.gz


In [4]:
# 初始化 权重 与 偏置
def weight_variable(shape):
    '''
        shape: 1-D 整型tensor 或python array
    '''
    # truncated_normal 生成截断正态分布随机数 即 所有随机数均落在(\mu+-2sigma)内
    
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    return tf.Variable(tf.ones(shape)/10)
    

In [5]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [6]:
# x 特征
# y 真实label
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [7]:
# 定义卷积层

W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [8]:
# 全连接层 —— 1
# 由于经过两次 2*2池化 因此图像长宽变为 1/4 即7*7 
# 而由于有64个输出通道，因此输入节点数为 7*7*64
# 设第一层隐含节点数为 1024
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# 将池化结果展开
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(h_fc1)

# dropout 层
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# softmax 层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
h_fc2 = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_conv = tf.nn.softmax(h_fc2)

In [9]:
# 损失函数
loss = -tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=h_fc2))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [10]:
# 准确率
correct_prediction = tf.equal(tf.argmax(y_conv, axis=1), tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
sess.run(tf.global_variables_initializer())

for i in range(1,20001):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y:batch[1], keep_prob:1})
#         train_accuracy = accuracy.run(feed_dict= {x:batch[0], y:batch[1], keep_prob:1})
        print("step %d, train accuracy %f"% (i, train_accuracy))
    _ = sess.run(train, feed_dict={x:batch[0], y:batch[1], keep_prob:0.5})
#     train.run(feed_dict={x:batch[0], y:batch[1], keep_prob:0.5})

step 100, train accuracy 0.160000
step 200, train accuracy 0.060000
step 300, train accuracy 0.180000
step 400, train accuracy 0.100000
step 500, train accuracy 0.140000
step 600, train accuracy 0.140000
step 700, train accuracy 0.060000
step 800, train accuracy 0.140000
step 900, train accuracy 0.060000
step 1000, train accuracy 0.120000
step 1100, train accuracy 0.160000
step 1200, train accuracy 0.080000
step 1300, train accuracy 0.040000
step 1400, train accuracy 0.100000
step 1500, train accuracy 0.080000
step 1600, train accuracy 0.100000
step 1700, train accuracy 0.060000
step 1800, train accuracy 0.120000
step 1900, train accuracy 0.140000
step 2000, train accuracy 0.100000
step 2100, train accuracy 0.080000
step 2200, train accuracy 0.100000
step 2300, train accuracy 0.040000
step 2400, train accuracy 0.100000
step 2500, train accuracy 0.100000
step 2600, train accuracy 0.160000
step 2700, train accuracy 0.100000
step 2800, train accuracy 0.040000
step 2900, train accuracy 0.0

KeyboardInterrupt: 